In [96]:
#Author: Grant Parker
#Functions/packages/code taken, adapated, and modified from https://github.com/microsoft/nni/blob/36ba04c94e51eaa2a88b9e6e0a4077f3a9f93004/examples/compression/pruning/scheduled_pruning.py

#NNI packages
import nni
from nni.compression import TorchEvaluator
from nni.compression.pruning import TaylorPruner, AGPPruner
from nni.compression.utils import auto_set_denpendency_group_ids
from nni.compression.speedup import ModelSpeedup


#pytorch packages 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

#tensor packages 
from tensorboardX import SummaryWriter
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

from models import LeNet_5
from datasets import get_mnist_dataloaders

import models
import datasets
#import modelsNNI

import time

#imports from Models
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader
import inspect

#importing Resnet 
import torchvision.models as modelsRes 

In [97]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [98]:
#functions
def train(model: torch.nn.Module, optimizer: torch.optim.Optimizer, training_step,
          lr_scheduler: _LRScheduler, max_steps: int, max_epochs: int):
    assert max_epochs is not None or max_steps is not None
    #train_loader, test_loader = prepare_dataloader()
    #grant addittion ---
    batch_size = 100
    train_loader, test_loader = datasets.get_mnist_dataloaders(batch_size,batch_size)

    #----
    max_steps = max_steps if max_steps else max_epochs * len(train_loader)
    max_epochs = max_steps // len(train_loader) + (0 if max_steps % len(train_loader) == 0 else 1)
    count_steps = 0

    model.train()
    for epoch in range(max_epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            loss = training_step((data, target), model)
            loss.backward()
            optimizer.step()
            count_steps += 1
            if count_steps >= max_steps:
                acc = evaluate(model, test_loader)
                print(f'[Training Epoch {epoch} / Step {count_steps}] Final Acc: {acc}%')
                return
        acc = evaluate(model, test_loader)
        print(f'[Training Epoch {epoch} / Step {count_steps}] Final Acc: {acc}%')

def evaluate(model: torch.nn.Module, test_loader):
    model.eval()
    correct = 0.0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    return 100 * correct / len(test_loader.dataset)


def training_step(batch, model: torch.nn.Module):
    output = model(batch[0])
    loss = F.cross_entropy(output, batch[1])
    return loss

def prepare_optimizer(model: torch.nn.Module):
    optimize_params = [param for param in model.parameters() if param.requires_grad == True]
    optimizer = nni.trace(Adam)(optimize_params, lr=0.01)
    return optimizer

In [99]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if __name__ == '__main__':
    print("start")
    #grants additions 

    model = models.LeNet_5().to(device)
    #model = modelsRes.resnet18().to(device)
    #optimizer = optim.NAdam(model.parameters()) -- did not use this 
    optimizer = prepare_optimizer(model)
    batch_size = 100
    train(model, optimizer, training_step, lr_scheduler=None, max_steps=None, max_epochs=50)

    train_loader, test_loader = datasets.get_mnist_dataloaders(batch_size, batch_size)
    print('Original model paramater number: ', sum([param.numel() for param in model.parameters()]))
    print('Original model after 10 epochs finetuning acc: ', evaluate(model, test_loader), '%')

    

start
[Training Epoch 0 / Step 600] Final Acc: 98.08%
[Training Epoch 1 / Step 1200] Final Acc: 98.53%
[Training Epoch 2 / Step 1800] Final Acc: 98.38%
[Training Epoch 3 / Step 2400] Final Acc: 98.56%
[Training Epoch 4 / Step 3000] Final Acc: 98.49%
[Training Epoch 5 / Step 3600] Final Acc: 98.36%
[Training Epoch 6 / Step 4200] Final Acc: 98.41%
[Training Epoch 7 / Step 4800] Final Acc: 98.44%
[Training Epoch 8 / Step 5400] Final Acc: 98.25%
[Training Epoch 9 / Step 6000] Final Acc: 98.33%
Original model paramater number:  61706
Original model after 10 epochs finetuning acc:  98.33 %


In [100]:
target_sparsity = .95
#config_list = [{
    #'op_types': ['Conv2d'],
    #'sparse_ratio':target_sparsity
#}]
#config_list = [{
    #'op_names': ['conv1', 'conv2', 'fc3', 'fc4'],
    #'sparse_ratio': target_sparsity,
    #'global_group_id': 'fourLayers'
#}]
""" config_list = [{
    'op_names': ['conv1'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}, {
    'op_names': ['conv2'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}, {
    'op_names': ['fc3'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}, {
    'op_names': ['fc4'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}] """
config_list = [{
    'op_names': ['conv1', 'conv2'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}, {
    'op_names': ['fc3','fc4'],
    'global_group_id': 'fourLayers',
    'sparse_ratio': target_sparsity,
}]


dummy_input = torch.rand(6,1,28,28)
config_list = auto_set_denpendency_group_ids(model, config_list, dummy_input)
optimizer = prepare_optimizer(model)
evaluator = TorchEvaluator(train, optimizer, training_step)

sub_pruner = TaylorPruner(model,config_list, evaluator, training_steps=100)
scheduled_pruner = AGPPruner(sub_pruner, interval_steps=100, total_times=10)

_, masks = scheduled_pruner.compress(max_steps=100 * 10, max_epochs=None)
scheduled_pruner.unwrap_model()

model = ModelSpeedup(model, dummy_input, masks).speedup_model()

[Training Epoch 0 / Step 600] Final Acc: 48.34%
[Training Epoch 1 / Step 1000] Final Acc: 70.8%
[2023-12-06 20:01:13] Start to speedup the model...
[2023-12-06 20:01:13] Resolve the mask conflict before mask propagate...
[2023-12-06 20:01:13] dim0 sparsity: 0.909091
[2023-12-06 20:01:13] dim1 sparsity: 0.000000
0 Filter
[2023-12-06 20:01:13] dim0 sparsity: 0.909091
[2023-12-06 20:01:13] dim1 sparsity: 0.000000
[2023-12-06 20:01:13] Infer module masks...
[2023-12-06 20:01:13] Propagate original variables
[2023-12-06 20:01:13] Propagate variables for placeholder: x, output mask:  0.0000 
[2023-12-06 20:01:13] Propagate variables for call_module: conv1, weight:  0.8333 bias:  0.8333 , output mask:  0.0000 
[2023-12-06 20:01:13] Propagate variables for call_function: relu, output mask:  0.0000 
[2023-12-06 20:01:13] Propagate variables for call_function: max_pool2d, output mask:  0.0000 
[2023-12-06 20:01:13] Propagate variables for call_module: conv2, weight:  0.9375 bias:  0.9375 , outpu

In [101]:
print('Pruned model paramater number: ', sum([param.numel() for param in model.parameters()]))
print(model)

forward_source = inspect.getsource(model.forward)
print(forward_source)

Pruned model paramater number:  303
LeNet_5(
  (conv1): Conv2d(1, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(1, 1, kernel_size=(5, 5), stride=(1, 1))
  (fc3): Linear(in_features=25, out_features=6, bias=True)
  (fc4): Linear(in_features=6, out_features=5, bias=True)
  (fc5): Linear(in_features=5, out_features=10, bias=True)
)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.fc3(x.view(-1, 16* 5 * 5)))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)

        return x



In [102]:
import math

class LeNet_5Pruned(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, int(math.ceil(6-(6*target_sparsity))), 5, padding=2)
        self.conv2 = nn.Conv2d(int(math.ceil(6-(6*target_sparsity))), int(math.ceil(16-(16*target_sparsity))), 5)
        self.fc3 = nn.Linear(int(math.ceil(16-(16*target_sparsity))) * 5 * 5, int(math.ceil(120-(120*target_sparsity))))
        self.fc4 = nn.Linear(int(math.ceil(120-(120*target_sparsity))), int(math.ceil(84-(84*target_sparsity))))
        self.fc5 = nn.Linear(int(math.ceil(84-(84*target_sparsity))), 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.fc3(x.view(-1, int(math.ceil(16-(16*target_sparsity))) * 5 * 5)))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)

        return x

    

In [103]:
modelPruned = LeNet_5Pruned().to(device)

print(modelPruned)

forward_source = inspect.getsource(modelPruned.forward)
print(forward_source)

LeNet_5Pruned(
  (conv1): Conv2d(1, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(1, 1, kernel_size=(5, 5), stride=(1, 1))
  (fc3): Linear(in_features=25, out_features=6, bias=True)
  (fc4): Linear(in_features=6, out_features=5, bias=True)
  (fc5): Linear(in_features=5, out_features=10, bias=True)
)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.fc3(x.view(-1, int(math.ceil(16-(16*target_sparsity))) * 5 * 5)))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)

        return x



In [104]:
optimizer = prepare_optimizer(modelPruned)
train(modelPruned, optimizer, training_step, lr_scheduler=None, max_steps=None, max_epochs=50)

[Training Epoch 0 / Step 600] Final Acc: 87.41%
[Training Epoch 1 / Step 1200] Final Acc: 88.83%
[Training Epoch 2 / Step 1800] Final Acc: 89.68%
[Training Epoch 3 / Step 2400] Final Acc: 89.89%
[Training Epoch 4 / Step 3000] Final Acc: 89.19%
[Training Epoch 5 / Step 3600] Final Acc: 89.72%
[Training Epoch 6 / Step 4200] Final Acc: 90.48%
[Training Epoch 7 / Step 4800] Final Acc: 89.94%
[Training Epoch 8 / Step 5400] Final Acc: 90.49%
[Training Epoch 9 / Step 6000] Final Acc: 89.0%


In [105]:
print('Pruned model after 10 epochs finetuning acc: ', evaluate(modelPruned, test_loader), '%')

Pruned model after 10 epochs finetuning acc:  89.0 %
